In [1]:
# DATA2001 Assignment

In [2]:
#First, we will begin by importing the various libraries.

In [3]:
## Importing the libraries
import pandas as pd
import matplotlib as plot
import numpy as np
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#Let us first take a look at what the csv files look like.

In [ ]:
pd.read_csv('Neighbourhoods.csv')

In [4]:
pd.read_csv('BusinessStats.csv')

,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
...,...,...,...,...,...,...,...,...,...
2296,901011001,Christmas Island,0,0,0,0,0,0,0
2297,901021002,Cocos (Keeling) Islands,7,3,0,0,0,0,0
2298,901031003,Jervis Bay,6,0,3,0,0,0,0
2299,901041004,Norfolk Island,0,0,0,0,0,0,0


In [5]:
pd.read_csv('StatisticalAreas.csv')

,area_id,area_name,parent_area_id
0,1,New South Wales,0
1,10,Greater Sydney,1
2,11,Rest of NSW,1
3,2,Victoria,0
4,20,Greater Melbourne,2
...,...,...,...
429,106,Hunter Valley exc Newcastle,11
430,11102,Lake Macquarie - West,111
431,111,Newcastle and Lake Macquarie,11
432,11402,Southern Highlands,114


In [6]:
### Now let's open the shapefile
RFSNSW_BPL = gpd.read_file('/Users/sidath/Documents/4. DATA2001/Assignment/Datasets/RFSNSW_BFPL/RFSNSW_BFPL.shp')


NameError: name 'gpd' is not defined

In [7]:
print(RFSNSW_BPL)

NameError: name 'RFSNSW_BPL' is not defined

In [8]:
#Now, we want to upload this shp file to pgAdmin.

In [9]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

data_path = "."

def pgconnect(credential_filepath, db_schema="public"):
    # please replace <your_unikey> and <your_SID> with your own details in the credentials file!
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        HOST       = db_conn_dict['host']
        DB_USER    = db_conn_dict['user']
        DB_PW      = db_conn_dict['password']
        DEFAULT_DB = db_conn_dict['user']

        try:
            db = create_engine('postgres+psycopg2://'+DB_USER+':'+DB_PW+'@'+HOST+'/'+DEFAULT_DB, echo=False)
            conn = db.connect()
            print('connected')
        except Exception as e:
            print("unable to connect to the database")
            print(e)
        return db,conn

credfilepath = os.path.join(data_path, "data2x01_db.json")
db, conn = pgconnect(credfilepath)

print(db.table_names())

connected
['spatial_ref_sys', 'organisations', 'statisticalareas']


In [10]:
#Now, we want to upload our files to PgAdmin.

In [11]:
# statistical_areas = pd.read_csv('StatisticalAreas.csv')

In [12]:
# if you want to reset the table
conn.execute("DROP TABLE IF EXISTS statisticalareas")

# 2nd: ensure that the schema is in place
statistical_areas = """CREATE TABLE IF NOT EXISTS statisticalareas (
                         area_id      Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         parent_area_id Integer
                   )"""
conn.execute(statistical_areas)
print(db.table_names())

['spatial_ref_sys', 'organisations', 'statisticalareas']


In [13]:
statistical_areas = pd.read_csv('StatisticalAreas.csv')

table_name = "statisticalareas"
statistical_areas.to_sql(table_name, con=conn, if_exists='replace',index=False)

In [14]:
res = pd.read_sql_query('SELECT COUNT(*) FROM statisticalareas', conn)
res

,count
0,434


In [15]:
res = pd.read_sql_query("SELECT * FROM statisticalareas",conn)
res

,area_id,area_name,parent_area_id
0,1,New South Wales,0
1,10,Greater Sydney,1
2,11,Rest of NSW,1
3,2,Victoria,0
4,20,Greater Melbourne,2
...,...,...,...
429,106,Hunter Valley exc Newcastle,11
430,11102,Lake Macquarie - West,111
431,111,Newcastle and Lake Macquarie,11
432,11402,Southern Highlands,114
